In [71]:
import random 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier as XGBC
import plotly.express as px
from umap import UMAP
from sklearn.metrics import f1_score, balanced_accuracy_score
from plotly.subplots import make_subplots
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, auc
from IPython.display import Image, display
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from plotly.subplots import make_subplots


In [72]:
base_path = 'output'

X_train = pd.read_csv(f'{base_path}/X_train_3.csv',index_col=0)
y_train = pd.read_csv(f'{base_path}/y_train_3.csv',index_col=0)
y_train = y_train.squeeze()
X_val = pd.read_csv(f'{base_path}/X_val_3.csv',index_col=0)
y_val = pd.read_csv(f'{base_path}/y_val_3.csv',index_col=0)
y_val = y_val.squeeze()
X_test = pd.read_csv(f'{base_path}/X_test_3.csv',index_col=0)
y_test = pd.read_csv(f'{base_path}/y_test_3.csv',index_col=0)
y_test = y_test.squeeze()


In [73]:

X_train = pd.concat([X_train, X_test], axis=0)
y_train = pd.concat([y_train, y_test], axis=0)

X_test=X_val
y_test=y_val


In [74]:
X_test.shape

(1940, 63)

 Random Forest

In [75]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report


In [126]:
rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')

rf.fit(X_train, y_train)
model = SelectFromModel(rf, prefit=True)
X_train_selected = model.transform(X_train)
X_test_selected = model.transform(X_test)
selected_feature = X_train.columns[model.get_support()].tolist()


param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


stratifiedCV = StratifiedKFold(n_splits=5)


BestParams = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42, class_weight='balanced'),
    param_grid=param_grid,
    cv=stratifiedCV ,
    scoring='f1_weighted',  
    n_jobs=-1
)
BestParams.fit(X_train_selected, y_train)


best_rf =  BestParams.best_estimator_

y_pred_test = best_rf.predict(X_test_selected)
report = classification_report(y_test, y_pred_test)
print(report)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred_test)
print("Balanced Accuracy:", balanced_accuracy)
weighted_f1 = f1_score(y_test, y_pred_test, average='weighted')
print("Weighted F1 Score:", weighted_f1)


y_pred_train = best_rf.predict(X_train_selected)
report = classification_report(y_train, y_pred_train)
print(report)
balanced_accuracy = balanced_accuracy_score(y_train, y_pred_train)
print("Balanced Accuracy:", balanced_accuracy)
weighted_f1 = f1_score(y_train, y_pred_train, average='weighted')
print("Weighted F1 Score:", weighted_f1)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning:

X has feature names, but SelectFromModel was fitted without feature names

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning:

X has feature names, but SelectFromModel was fitted without feature names



              precision    recall  f1-score   support

           0       0.33      0.31      0.32       539
           1       0.37      0.33      0.35       658
           2       0.47      0.53      0.50       743

    accuracy                           0.40      1940
   macro avg       0.39      0.39      0.39      1940
weighted avg       0.40      0.40      0.40      1940

Balanced Accuracy: 0.39086489538530267
Weighted F1 Score: 0.3979721097927787
              precision    recall  f1-score   support

           0       0.87      0.90      0.89      2155
           1       0.95      0.90      0.93      2632
           2       0.88      0.90      0.89      2969

    accuracy                           0.90      7756
   macro avg       0.90      0.90      0.90      7756
weighted avg       0.90      0.90      0.90      7756

Balanced Accuracy: 0.9002639435473131
Weighted F1 Score: 0.9004006116056693


In [127]:
rf_cm = confusion_matrix(y_test, y_pred_test)

In [128]:
rf_cm

array([[167, 172, 200],
       [194, 217, 247],
       [151, 196, 396]])

In [142]:

class_labels = ['1 to 2 days', '2 to 4 days', '4+ days']

fig = go.Figure(data=go.Heatmap(
    z=rf_cm[::-1],
    x=class_labels,
    y=class_labels[::-1],
    colorscale='Blues',
    showscale=True
))

annotations = []
for i, row in enumerate(rf_cm):
    for j, value in enumerate(row):
        annotations.append(
            go.layout.Annotation(
                text=str(value),
                x=class_labels[j],
                y=class_labels[i],
                showarrow=False,
                font=dict(color="white" if rf_cm[i][j] > max(rf_cm[0]) else "black")
            )
        )

fig.update_layout(
    title={
        'text': 'Confusion Matrix for 3 Classes (Random Forest)',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    annotations=annotations,
    xaxis=dict(title='Predicted label'),
    yaxis=dict(title='Actual label')
)


fig.update_layout(width=640, height=516)

fig.show()


Logistic Regression

In [145]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000,multi_class='multinomial', solver='lbfgs',class_weight='balanced')
model.fit(X_train, y_train)


y_pred_train = model.predict(X_train)
report = classification_report(y_train, y_pred_train)
print(report)
balanced_accuracy = balanced_accuracy_score(y_train, y_pred_train)
print("Balanced Accuracy:", balanced_accuracy)
weighted_f1 = f1_score(y_train, y_pred_train, average='weighted')
print("Weighted F1 Score:", weighted_f1)




y_pred_test = model.predict(X_test)
report = classification_report(y_test, y_pred_test)
print(report)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred_test)
print("Balanced Accuracy:", balanced_accuracy)
weighted_f1 = f1_score(y_test, y_pred_test, average='weighted')
print("Weighted F1 Score:", weighted_f1)


              precision    recall  f1-score   support

           0       0.35      0.43      0.38      2155
           1       0.39      0.28      0.33      2632
           2       0.48      0.51      0.49      2969

    accuracy                           0.41      7756
   macro avg       0.40      0.41      0.40      7756
weighted avg       0.41      0.41      0.41      7756

Balanced Accuracy: 0.4071863986030324
Weighted F1 Score: 0.40635303330949096
              precision    recall  f1-score   support

           0       0.28      0.34      0.31       539
           1       0.37      0.30      0.33       658
           2       0.46      0.47      0.47       743

    accuracy                           0.38      1940
   macro avg       0.37      0.37      0.37      1940
weighted avg       0.38      0.38      0.38      1940

Balanced Accuracy: 0.3701607907038941
Weighted F1 Score: 0.37646689744321754


In [146]:
lr_cm = confusion_matrix(y_test, y_pred_test)
lr_cm

array([[184, 166, 189],
       [245, 197, 216],
       [220, 174, 349]])

In [147]:

class_labels = ['1 to 2 days', '2 to 4 days', '4+ days']

fig = go.Figure(data=go.Heatmap(
    z=lr_cm[::-1],
    x=class_labels,
    y=class_labels[::-1],
    colorscale='Blues',
    showscale=True
))

annotations = []
for i, row in enumerate(lr_cm):
    for j, value in enumerate(row):
        annotations.append(
            go.layout.Annotation(
                text=str(value),
                x=class_labels[j],
                y=class_labels[i],
                showarrow=False,
                font=dict(color="white" if lr_cm[i][j] > max(lr_cm[0]) else "black")
            )
        )

fig.update_layout(
    title={
        'text': 'Confusion Matrix for 3 Classes (Logistic Regression)',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    annotations=annotations,
    xaxis=dict(title='Predicted label'),
    yaxis=dict(title='Actual label')
)


fig.update_layout(width=640, height=516)

fig.show()


Decision Tree

In [148]:
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'class_weight': ['balanced']
}
stratifiedCV = StratifiedKFold(n_splits=5)
BestParams = GridSearchCV(
    estimator=DecisionTreeClassifier(random_state=42),
    param_grid=param_grid,
    cv=stratifiedCV ,
    scoring='f1_weighted',
    n_jobs=-1
)
BestParams.fit(X_train, y_train)
best_dt = BestParams.best_estimator_
selector = SelectFromModel(estimator=best_dt, prefit=True)
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)
selected_features = X_train.columns[selector.get_support()]
best_dt.fit(X_train_selected, y_train)
y_pred_test = best_dt.predict(X_test_selected)
report = classification_report(y_test, y_pred_test)
print(report)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred_test)
print("Balanced Accuracy:", balanced_accuracy)
weighted_f1 = f1_score(y_test, y_pred_test, average='weighted')
print("Weighted F1 Score:", weighted_f1)



y_pred_train = best_dt.predict(X_train_selected)
report = classification_report(y_train, y_pred_train)
print(report)
balanced_accuracy = balanced_accuracy_score(y_train, y_pred_train)
print("Balanced Accuracy:", balanced_accuracy)
weighted_f1 = f1_score(y_train, y_pred_train, average='weighted')
print("Weighted F1 Score:", weighted_f1)


              precision    recall  f1-score   support

           0       0.30      0.32      0.31       539
           1       0.35      0.35      0.35       658
           2       0.44      0.42      0.43       743

    accuracy                           0.37      1940
   macro avg       0.36      0.36      0.36      1940
weighted avg       0.37      0.37      0.37      1940

Balanced Accuracy: 0.3629735016170521
Weighted F1 Score: 0.36899434038608536
              precision    recall  f1-score   support

           0       0.54      0.57      0.55      2155
           1       0.53      0.56      0.55      2632
           2       0.58      0.53      0.56      2969

    accuracy                           0.55      7756
   macro avg       0.55      0.55      0.55      7756
weighted avg       0.55      0.55      0.55      7756

Balanced Accuracy: 0.5531524437316114
Weighted F1 Score: 0.551734872385199


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning:

X has feature names, but SelectFromModel was fitted without feature names

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning:

X has feature names, but SelectFromModel was fitted without feature names



In [150]:
dt_cm = confusion_matrix(y_test, y_pred_test)

In [151]:
dt_cm

array([[172, 202, 165],
       [189, 232, 237],
       [211, 222, 310]])

In [152]:

class_labels = ['1 to 2 days', '2 to 4 days', '4+ days']

fig = go.Figure(data=go.Heatmap(
    z=dt_cm[::-1],
    x=class_labels,
    y=class_labels[::-1],
    colorscale='Blues',
    showscale=True
))

annotations = []
for i, row in enumerate(dt_cm):
    for j, value in enumerate(row):
        annotations.append(
            go.layout.Annotation(
                text=str(value),
                x=class_labels[j],
                y=class_labels[i],
                showarrow=False,
                font=dict(color="white" if dt_cm[i][j] > max(dt_cm[0]) else "black")
            )
        )

fig.update_layout(
    title={
        'text': 'Confusion Matrix for 3 Classes (Decision Tree)',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    annotations=annotations,
    xaxis=dict(title='Predicted label'),
    yaxis=dict(title='Actual label')
)


fig.update_layout(width=640, height=516)

fig.show()


Adaboost

In [153]:
ada = AdaBoostClassifier(n_estimators=100, random_state=42)
ada.fit(X_train, y_train)
model = SelectFromModel(ada, prefit=True)
X_train_selected = model.transform(X_train)
X_test_selected = model.transform(X_test)
selected_feature= X_train.columns[model.get_support()].tolist()
param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.1, 0.5, 1]
}
stratifiedCV = StratifiedKFold(n_splits=5)
BestParams = GridSearchCV(
    estimator=AdaBoostClassifier(random_state=42),
    param_grid=param_grid,
    cv=stratifiedCV ,
    scoring='f1_weighted',
    n_jobs=-1
)
BestParams.fit(X_train_selected, y_train)
best_ada =  BestParams.best_estimator_
y_pred_test = best_ada.predict(X_test_selected)
report = classification_report(y_test, y_pred_test)
print(report)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred_test)
print("Balanced Accuracy:", balanced_accuracy)
weighted_f1 = f1_score(y_test, y_pred_test, average='weighted')
print("Weighted F1 Score:", weighted_f1)


y_pred_train = best_ada.predict(X_train_selected)
report = classification_report(y_train, y_pred_train)
print(report)
balanced_accuracy = balanced_accuracy_score(y_train, y_pred_train)
print("Balanced Accuracy:", balanced_accuracy)
weighted_f1 = f1_score(y_train, y_pred_train, average='weighted')
print("Weighted F1 Score:", weighted_f1)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning:

The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning:

X has feature names, but SelectFromModel was fitted without feature names

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning:

X has feature names, but SelectFromModel was fitted without feature names

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/s

              precision    recall  f1-score   support

           0       0.33      0.18      0.24       539
           1       0.36      0.35      0.35       658
           2       0.44      0.60      0.51       743

    accuracy                           0.40      1940
   macro avg       0.38      0.38      0.37      1940
weighted avg       0.38      0.40      0.38      1940

Balanced Accuracy: 0.37597642648021673
Weighted F1 Score: 0.3799945676323697
              precision    recall  f1-score   support

           0       0.46      0.22      0.30      2155
           1       0.43      0.41      0.42      2632
           2       0.48      0.67      0.56      2969

    accuracy                           0.46      7756
   macro avg       0.45      0.44      0.43      7756
weighted avg       0.46      0.46      0.44      7756

Balanced Accuracy: 0.4350327899635049
Weighted F1 Score: 0.43885562237326065


In [154]:
ada_cm = confusion_matrix(y_test, y_pred_test)
ada_cm 

array([[ 99, 199, 241],
       [115, 229, 314],
       [ 88, 212, 443]])

In [155]:

class_labels = ['1 to 2 days', '2 to 4 days', '4+ days']

fig = go.Figure(data=go.Heatmap(
    z=ada_cm[::-1],
    x=class_labels,
    y=class_labels[::-1],
    colorscale='Blues',
    showscale=True
))

annotations = []
for i, row in enumerate(ada_cm):
    for j, value in enumerate(row):
        annotations.append(
            go.layout.Annotation(
                text=str(value),
                x=class_labels[j],
                y=class_labels[i],
                showarrow=False,
                font=dict(color="white" if ada_cm[i][j] > max(ada_cm[0]) else "black")
            )
        )

fig.update_layout(
    title={
        'text': 'Confusion Matrix for 3 Classes (Adaboost)',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    annotations=annotations,
    xaxis=dict(title='Predicted label'),
    yaxis=dict(title='Actual label')
)


fig.update_layout(width=640, height=516)

fig.show()


XGboost

In [156]:
from xgboost import XGBClassifier

xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb.fit(X_train, y_train)
model = SelectFromModel(xgb, prefit=True)
X_train_selected = model.transform(X_train)
X_test_selected = model.transform(X_test)
selected_feature= X_train.columns[model.get_support()].tolist()
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}
stratifiedCV = StratifiedKFold(n_splits=5)
BestParams = GridSearchCV(
    estimator=XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    param_grid=param_grid,
    cv=stratifiedCV ,
    scoring='f1_weighted',
    n_jobs=-1
)
BestParams.fit(X_train_selected, y_train)
best_xgb = BestParams.best_estimator_
y_pred_test = best_xgb.predict(X_test_selected)
report = classification_report(y_test, y_pred_test)
print(report)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred_test)
print("Balanced Accuracy:", balanced_accuracy)
weighted_f1 = f1_score(y_test, y_pred_test, average='weighted')
print("Weighted F1 Score:", weighted_f1)



y_pred_train = best_xgb.predict(X_train_selected)
report = classification_report(y_train, y_pred_train)
print(report)
balanced_accuracy = balanced_accuracy_score(y_train, y_pred_train)
print("Balanced Accuracy:", balanced_accuracy)
weighted_f1 = f1_score(y_train, y_pred_train, average='weighted')
print("Weighted F1 Score:", weighted_f1)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning:

X has feature names, but SelectFromModel was fitted without feature names

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning:

X has feature names, but SelectFromModel was fitted without feature names



              precision    recall  f1-score   support

           0       0.35      0.21      0.26       539
           1       0.39      0.35      0.37       658
           2       0.46      0.64      0.54       743

    accuracy                           0.42      1940
   macro avg       0.40      0.40      0.39      1940
weighted avg       0.41      0.42      0.40      1940

Balanced Accuracy: 0.3997249857559046
Weighted F1 Score: 0.4038819538566423
              precision    recall  f1-score   support

           0       0.71      0.42      0.53      2155
           1       0.61      0.59      0.60      2632
           2       0.59      0.79      0.67      2969

    accuracy                           0.62      7756
   macro avg       0.64      0.60      0.60      7756
weighted avg       0.63      0.62      0.61      7756

Balanced Accuracy: 0.5971880121612597
Weighted F1 Score: 0.6075084120502319


In [157]:
xgb_cm = confusion_matrix(y_test, y_pred_test)
xgb_cm

array([[114, 183, 242],
       [121, 231, 306],
       [ 91, 179, 473]])

In [158]:

class_labels = ['1 to 2 days', '2 to 4 days', '4+ days']

fig = go.Figure(data=go.Heatmap(
    z=xgb_cm[::-1],
    x=class_labels,
    y=class_labels[::-1],
    colorscale='Blues',
    showscale=True
))

annotations = []
for i, row in enumerate(xgb_cm):
    for j, value in enumerate(row):
        annotations.append(
            go.layout.Annotation(
                text=str(value),
                x=class_labels[j],
                y=class_labels[i],
                showarrow=False,
                font=dict(color="white" if xgb_cm[i][j] > max(xgb_cm[0]) else "black")
            )
        )

fig.update_layout(
    title={
        'text': 'Confusion Matrix for 3 Classes (XGboost)',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    annotations=annotations,
    xaxis=dict(title='Predicted label'),
    yaxis=dict(title='Actual label')
)


fig.update_layout(width=640, height=516)

fig.show()


SVM

In [159]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split


model = SVC(decision_function_shape='ovo',class_weight='balanced')  
model.fit(X_train, y_train)


print(model.score(X_test, y_test))
y_pred = model.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
print("Balanced Accuracy:", balanced_accuracy)
weighted_f1 = f1_score(y_test, y_pred, average='weighted')
print("Weighted F1 Score:", weighted_f1)



y_pred_train = model.predict(X_train)
report = classification_report(y_train, y_pred_train)
print(report)
balanced_accuracy = balanced_accuracy_score(y_train, y_pred_train)
print("Balanced Accuracy:", balanced_accuracy)
weighted_f1 = f1_score(y_train, y_pred_train, average='weighted')
print("Weighted F1 Score:", weighted_f1)


0.3974226804123711
              precision    recall  f1-score   support

           0       0.33      0.41      0.37       539
           1       0.36      0.33      0.34       658
           2       0.50      0.45      0.47       743

    accuracy                           0.40      1940
   macro avg       0.40      0.40      0.39      1940
weighted avg       0.41      0.40      0.40      1940

Balanced Accuracy: 0.3961620094278708
Weighted F1 Score: 0.3991612635031508
              precision    recall  f1-score   support

           0       0.58      0.70      0.64      2155
           1       0.64      0.59      0.62      2632
           2       0.69      0.64      0.67      2969

    accuracy                           0.64      7756
   macro avg       0.64      0.64      0.64      7756
weighted avg       0.65      0.64      0.64      7756

Balanced Accuracy: 0.6440540788466246
Weighted F1 Score: 0.640597789924762


In [160]:
svm_cm = confusion_matrix(y_test, y_pred)
svm_cm 

array([[223, 180, 136],
       [249, 214, 195],
       [202, 207, 334]])

In [161]:

class_labels = ['1 to 2 days', '2 to 4 days', '4+ days']

fig = go.Figure(data=go.Heatmap(
    z=svm_cm[::-1],
    x=class_labels,
    y=class_labels[::-1],
    colorscale='Blues',
    showscale=True
))

annotations = []
for i, row in enumerate(svm_cm):
    for j, value in enumerate(row):
        annotations.append(
            go.layout.Annotation(
                text=str(value),
                x=class_labels[j],
                y=class_labels[i],
                showarrow=False,
                font=dict(color="white" if svm_cm[i][j] > max(svm_cm[0]) else "black")
            )
        )

fig.update_layout(
    title={
        'text': 'Confusion Matrix for 3 Classes (SVM)',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    annotations=annotations,
    xaxis=dict(title='Predicted label'),
    yaxis=dict(title='Actual label')
)


fig.update_layout(width=640, height=516)

fig.show()


kNN

In [162]:

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectFromModel


knn = KNeighborsClassifier()


rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf.fit(X_train, y_train)
model = SelectFromModel(rf, prefit=True)
X_train_selected = model.transform(X_train)
X_test_selected = model.transform(X_test)
selected_feature = X_train.columns[model.get_support()].tolist()


param_grid = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}


stratifiedCV = StratifiedKFold(n_splits=5)


BestParams = GridSearchCV(
    estimator=KNeighborsClassifier(),
    param_grid=param_grid,
    cv=stratifiedCV,
    scoring='f1_weighted',
    n_jobs=-1
)
BestParams.fit(X_train_selected, y_train)


best_knn = BestParams.best_estimator_


y_pred_test = best_knn.predict(X_test_selected)
report = classification_report(y_test, y_pred_test)
print(report)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred_test)
print("Balanced Accuracy:", balanced_accuracy)
weighted_f1 = f1_score(y_test, y_pred_test, average='weighted')
print("Weighted F1 Score:", weighted_f1)


y_pred_train = best_knn.predict(X_train_selected)
report = classification_report(y_train, y_pred_train)
print(report)
balanced_accuracy = balanced_accuracy_score(y_train, y_pred_train)
print("Balanced Accuracy:", balanced_accuracy)
weighted_f1 = f1_score(y_train, y_pred_train, average='weighted')
print("Weighted F1 Score:", weighted_f1)



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning:

X has feature names, but SelectFromModel was fitted without feature names

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning:

X has feature names, but SelectFromModel was fitted without feature names



              precision    recall  f1-score   support

           0       0.32      0.28      0.30       539
           1       0.38      0.37      0.37       658
           2       0.41      0.45      0.43       743

    accuracy                           0.38      1940
   macro avg       0.37      0.37      0.37      1940
weighted avg       0.37      0.38      0.37      1940

Balanced Accuracy: 0.3662681365946286
Weighted F1 Score: 0.3728405049770856
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2155
           1       1.00      1.00      1.00      2632
           2       1.00      1.00      1.00      2969

    accuracy                           1.00      7756
   macro avg       1.00      1.00      1.00      7756
weighted avg       1.00      1.00      1.00      7756

Balanced Accuracy: 1.0
Weighted F1 Score: 1.0


In [163]:
knn_cm = confusion_matrix(y_test, y_pred_test)
knn_cm 

array([[151, 171, 217],
       [141, 242, 275],
       [182, 226, 335]])

In [164]:

class_labels = ['1 to 2 days', '2 to 4 days', '4+ days']

fig = go.Figure(data=go.Heatmap(
    z=knn_cm[::-1],
    x=class_labels,
    y=class_labels[::-1],
    colorscale='Blues',
    showscale=True
))

annotations = []
for i, row in enumerate(knn_cm):
    for j, value in enumerate(row):
        annotations.append(
            go.layout.Annotation(
                text=str(value),
                x=class_labels[j],
                y=class_labels[i],
                showarrow=False,
                font=dict(color="white" if knn_cm[i][j] > max(knn_cm[0]) else "black")
            )
        )

fig.update_layout(
    title={
        'text': 'Confusion Matrix for 3 Classes (kNN)',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    annotations=annotations,
    xaxis=dict(title='Predicted label'),
    yaxis=dict(title='Actual label')
)


fig.update_layout(width=640, height=516)

fig.show()


In [187]:

fig = make_subplots(rows=4, cols=2, subplot_titles=("Random Forest", "Logistic Regression",'Decision Tree',"Adaboost","XGboost", "SVM", "kNN") )


class_labels = ['1 to 2 days', '2 to 4 days', '4+ days']
z_min = min([min(map(min, cm)) for cm in [rf_cm, lr_cm, dt_cm, ada_cm, xgb_cm, svm_cm, knn_cm]])
z_max = max([max(map(max, cm)) for cm in [rf_cm, lr_cm, dt_cm, ada_cm, xgb_cm, svm_cm, knn_cm]])


fig.add_trace(
    go.Heatmap(
        z=rf_cm[::-1], 
        zmin=z_min,  
        zmax=z_max,
        x=class_labels, 
        y=class_labels[::-1], 
        colorscale='Blues',
        
        text=rf_cm[::-1],
        texttemplate="%{text}",
        textfont=dict(size=20),
        showscale=False,
    ),
    row=1, col=1

)

fig.add_trace(
    go.Heatmap(
        z=lr_cm[::-1], 
        zmin=z_min,  
        zmax=z_max,
        x=class_labels, 
        y=class_labels[::-1], 
        colorscale='Blues',
        text=lr_cm[::-1],
        texttemplate="%{text}",
        textfont=dict(size=20),
        showscale=False,
    ),
    row=1, col=2
)


fig.add_trace(
    go.Heatmap(
        z=dt_cm[::-1], 
        zmin=z_min,  
        zmax=z_max,
        x=class_labels, 
        y=class_labels[::-1], 
        colorscale='Blues',
        text=dt_cm[::-1],
        texttemplate="%{text}",
        textfont=dict(size=20),
        showscale=False,
    ),
    row=2, col=1
)



fig.add_trace(
    go.Heatmap(
        z=ada_cm[::-1], 
        zmin=z_min,  
        zmax=z_max,
        x=class_labels, 
        y=class_labels[::-1], 
        colorscale='Blues',
        text=ada_cm[::-1],
        texttemplate="%{text}",
        textfont=dict(size=20),
        showscale=False,
    ),
    row=2, col=2
)

fig.add_trace(
    go.Heatmap(
        z=xgb_cm[::-1], 
        zmin=z_min,  
        zmax=z_max,
        x=class_labels, 
        y=class_labels[::-1], 
        colorscale='Blues',
        text=xgb_cm[::-1],
        texttemplate="%{text}",
        textfont=dict(size=20),
        showscale=False,
    ),
    row=3, col=1
)
fig.add_trace(
    go.Heatmap(
        z=svm_cm[::-1], 
        zmin=z_min,  
        zmax=z_max,
        x=class_labels, 
        y=class_labels[::-1], 
        colorscale='Blues',
        text=svm_cm[::-1],
        texttemplate="%{text}",
        textfont=dict(size=20),
        showscale=False,
    ),
    row=3, col=2
)
fig.add_trace(
    go.Heatmap(
        z=knn_cm[::-1], 
        zmin=z_min,  
        zmax=z_max,
        x=class_labels, 
        y=class_labels[::-1], 
        colorscale='Blues',
        text=knn_cm[::-1],
        texttemplate="%{text}",
        textfont=dict(size=20),
        showscale=True,
    ),
    row=4, col=1
)

fig.update_yaxes(title_text="Actual label", row=1, col=1)
fig.update_yaxes(title_text="Actual label", row=2, col=1)
fig.update_yaxes(title_text="Actual label", row=3, col=1)
fig.update_yaxes(title_text="Actual label", row=4, col=1)
fig.update_yaxes(title_text="Actual label", row=1, col=2)
fig.update_yaxes(title_text="Actual label", row=2, col=2)
fig.update_yaxes(title_text="Actual label", row=3, col=2)

fig.update_xaxes(title_text="Predicted label", row=2, col=2)
fig.update_xaxes(title_text="Predicted label", row=2, col=1)
fig.update_xaxes(title_text="Predicted label", row=1, col=1)
fig.update_xaxes(title_text="Predicted label", row=1, col=2)
fig.update_xaxes(title_text="Predicted label", row=3, col=1)
fig.update_xaxes(title_text="Predicted label", row=3, col=2)
fig.update_xaxes(title_text="Predicted label", row=4, col=1)


for row in range(1, 5):
    for col in range(1, 3):
        fig.update_xaxes(tickfont=dict(size=15), row=row, col=col)
        fig.update_yaxes(tickfont=dict(size=15), row=row, col=col)
        fig.update_xaxes(title_font=dict(size=15), row=row, col=col)

fig.update_layout(
    width=1500, 
    height=2000,
    title_text="Confusion Matrices for 3 Classes",
    title_x=0.5,
    
    
)


fig.show()